# Template Library Basics

This template intentionally ships only workshop-agnostic helpers in `simulated_city`:

- Load configuration from `config.yaml` + optional `.env` (`simulated_city.config`)
- Build MQTT topics and connect/publish (`simulated_city.mqtt`)

The actual city simulation is an exercise (see `docs/exercises.md`).

In [ ]:
# Imports used in this notebook.
import json

from simulated_city.config import load_config
from simulated_city.mqtt import MqttConnector, MqttPublisher

In [ ]:
# Load and print settings from config.yaml (and .env for secrets, if present).
cfg = load_config()
print("MQTT broker:", f"{cfg.mqtt.host}:{cfg.mqtt.port}", "tls=", cfg.mqtt.tls)


In [ ]:
# Build an example topic + JSON payload (no network calls yet).
# Topic format: "simulated-city/{suffix}"
events_topic = "simulated-city/events/demo"
payload = json.dumps({"hello": "humtek"})

## Connect and publish

This section publishes **one** MQTT message so you can confirm your broker credentials work.


Tip: in many broker dashboards you’ll want to subscribe to `simulated-city/#` (or your configured base topic + `/#`) to see messages.

In [ ]:

# Create connector and publisher
connector = MqttConnector(cfg.mqtt, client_id_suffix="notebook")
connector.connect()

# Wait for connection before publishing
if connector.wait_for_connection(timeout=5.0):
    publisher = MqttPublisher(connector)
    result = publisher.publish_json(events_topic, payload)
    
    if result.rc == 0:
        print("Published successfully!")
        print("Topic:", events_topic)
        print("Payload:", payload)
    else:
        print(f"Publish failed with return code: {result.rc}")
        print("Result:", result)
else:
    print("Failed to connect to MQTT broker within timeout.")

connector.disconnect()


## CRS transforms (WGS84 ↔ EPSG:25832)

Most real-world locations are given as **WGS84** latitude/longitude (EPSG:4326).
For distance-based math (meters), it’s often easier to work in **UTM zone 32N** (EPSG:25832) in Denmark.

The next cell uses an approximate point near Copenhagen City Hall and converts:
1) WGS84 (lat/lon) → EPSG:25832 (Easting/Northing in meters)
2) EPSG:25832 → back to WGS84 (to show the round-trip error is tiny)

In [ ]:
# Convert a real-world WGS84 point (lat/lon) to UTM32 meters (EPSG:25832) and back.
#
# This cell shows TWO ways to do it:
# 1) The general `transform_xy(...)` function (more flexible, but you must remember axis order)
# 2) The convenience helpers `wgs2utm(...)` / `utm2wgs(...)` (recommended for beginners)

from simulated_city.geo import EPSG_25832, transform_xy, utm2wgs, wgs2utm

# Approximate point near Copenhagen City Hall (Rådhuspladsen).
lat, lon = 55.6761, 12.5683
print("Start (WGS84 lat, lon):", (lat, lon))


print("\n--- Version A: transform_xy (note axis order) ---")
# Note: transforms use (x, y) = (lon, lat) when converting to/from EPSG:4326.
e_a, n_a = transform_xy(lon, lat, from_crs="EPSG:4326", to_crs=EPSG_25832)
print("To EPSG:25832 (E, N) [m]:", (e_a, n_a))

lon_back_a, lat_back_a = transform_xy(e_a, n_a, from_crs=EPSG_25832, to_crs="EPSG:4326")
print("Back to WGS84 (lat, lon):", (lat_back_a, lon_back_a))

print("\n--- Version B: convenience helpers (beginner-friendly) ---")
e_b, n_b = wgs2utm(lat, lon)
print("To EPSG:25832 (E, N) [m]:", (e_b, n_b))

lat_back_b, lon_back_b = utm2wgs(e_b, n_b)
print("Back to WGS84 (lat, lon):", (lat_back_b, lon_back_b))

print("\n--- Compare results (A vs B) ---")
print("ΔE (m):", e_b - e_a)
print("ΔN (m):", n_b - n_a)
print("Round-trip error via helpers (degrees):", (lat_back_b - lat, lon_back_b - lon))


## Web map example (Copenhagen City Hall)

This section shows a point "in front of" Copenhagen City Hall on a web map.
We start with an approximate WGS84 lat/lon for the City Hall area.

Note: anymap-ts renders as HTML/JS output. Save it as an HTML file and open in your browser, or use JupyterLab for inline display.

anymap-ts is optional; if it's not installed, the cell prints install instructions.

For more anymap-ts features, see `docs/maplibre_anymap.md`.

In [ ]:
# Approximate point near Copenhagen City Hall (Rådhuspladsen).
# If you need an exact surveyed point later, replace these numbers.
lat, lng = 55.6761, 12.5683

# For the web map we don't need any CRS transforms: anymap-ts expects WGS84 (lon/lat).

from anymap_ts import Map

m = Map(center=(lng,lat ), zoom=18)

# m.add_marker(lng,lat, popup=f"WGS84 (lat, lon): {lat:.6f}, {lng:.6f}")
display(m)

In [ ]:
# Try different basemaps (uncomment one to use):
# m.add_basemap("OpenStreetMap.Mapnik")  # Standard OpenStreetMap colors
m.add_basemap("Esri.WorldImagery")     # Satellite/orthophoto imagery
# m.add_basemap("CartoDB.DarkMatter")    # Dark theme

In [ ]:
m.add_3d_buildings()

In [ ]:
#m.set_sky()
m.set_sky(sky_color="#229CDA", horizon_color="#F0E4D4")

## Layer visibility and transparency

You can control **layers you've added** with `set_visibility()` and `set_opacity()`:

**Important:** These methods work on layers added via `add_basemap()`, `add_geojson()`, etc., but **not** on the base style's built-in layers (like the default "background" layer).

To control the base background:
1. **Option A:** Start with a minimal/blank style when creating the map
2. **Option B:** Add basemaps on top and control those instead
3. **Option C:** Access the MapLibre instance directly (advanced)

In [ ]:
# Control layers you've ADDED (works):
# m.set_visibility("Esri.WorldImagery", False)  # Hide the basemap layer
# m.set_opacity("Esri.WorldImagery", 0.5)       # Make it semi-transparent

# Control base style layers (doesn't work via set_visibility):
# The default map style has its own "background" layer that can't be controlled this way.

# Workaround: Create a map with a blank style from the start
from curses import version

from anymap_ts import Map

# Example with minimal style (no default background):
m_blank = Map(
    center=(lng, lat),
    zoom=18,
    style={
        "version": 8,
         "sources": {},
        "layers": []
        }
    )
# Now add only the layers you want to control:
m_blank.add_basemap("Esri.WorldImagery")
m_blank.add_3d_buildings()
print("Tip: To fully control all layers, start with a blank style (see example above)")
display(m_blank)

## Layer ordering (z-order / stacking)

You can change the display order of layers with `move_layer()`:

- `move_layer(layer_id, before_id)` - moves `layer_id` to appear just before `before_id` in the stack
- `move_layer(layer_id)` - moves `layer_id` to the top (front) of the stack

Layers drawn later appear on top of earlier layers. This is useful when you need to control which data appears in front.

In [ ]:
# Example: Reorder layers in the blank style map
# The satellite imagery layer ("Esri.WorldImagery") is currently below the 3D buildings
# Let's move it to the top to see the effect:

m_blank.move_layer("Esri.WorldImagery")  # Move satellite to top (will cover buildings)

# Or move it before the 3D buildings layer:
# m_blank.move_layer("Esri.WorldImagery", "3d-buildings")  # Satellite below 3D buildings

print("Try uncommenting the move_layer() calls above to see layer reordering in action")

## Drawing tools and saving geometry

You can add a drawing control to the map for creating and editing geometry (points, lines, polygons) and save them as GeoJSON.

Use `add_draw_control()` to enable drawing tools, then `get_draw_data()` to retrieve drawn features and `save_draw_data()` to export to a file.

**Tip:** For the marker/point tool, click on the map to place points (don't drag). If markers appear transparent or won't stick:
1. Try clicking once to confirm placement instead of dragging
2. Use `get_draw_data()` and `save_draw_data()` to retrieve and save your drawn features to a GeoJSON file

In [ ]:
# Create a fresh map for drawing
from anymap_ts import Map
lat, lng = 55.6761, 12.5683
m_draw = Map(center=(lng, lat), zoom=18)
m_draw.add_basemap("Esri.WorldImagery")

# Add a control grid with specific tools
# Available controls (26 total): 'globe', 'fullscreen', 'north', 'terrain', 'search',
# 'viewState', 'inspect', 'vectorDataset', 'basemap', 'measure', 'geoEditor', 'bookmark',
# 'print', 'minimap', 'swipe', 'streetView', 'addVector', 'cogLayer', 'zarrLayer',
# 'pmtilesLayer', 'stacLayer', 'stacSearch', 'planetaryComputer', 'gaussianSplat', 'lidar', 'usgsLidar'

# Option A: Use all default controls
m_draw.add_control_grid(
    position="top-left",
    collapsed=True,
    title="Tools"
)

# Option B (commented out): Choose specific controls
# m_draw.add_control_grid(
#     default_controls=["search", "measure", "geoEditor", "basemap", "fullscreen"],
#     position="top-right",
#     collapsed=True
# )

# Add draw control with markers, lines, and polygons enabled
m_draw.add_draw_control()

display(m_draw)

In [ ]:
# Get the drawn features as GeoJSON (run this after drawing)
draw_data = m_draw.get_draw_data()
print(f"Drawn features: {len(draw_data['features'])} feature(s)")
print(draw_data)


In [ ]:
# Save drawn features to a GeoJSON file on your local drive
from pathlib import Path

# Create a data folder if it doesn't exist
output_dir = Path("./drawn_geometry")
output_dir.mkdir(exist_ok=True)

# Save the GeoJSON to a file
output_file = output_dir / "features.geojson"
m_draw.save_draw_data(output_file)
print(f"Saved {len(draw_data['features'])} feature(s) to {output_file}")
print(f"File location: {output_file.absolute()}")


In [ ]:
from anymap_ts import Map

m4 = Map(
    center=[-123.07, 44.05],
    zoom=15,
    pitch=60,
)
m4.add_basemap("CartoDB.DarkMatter")

# Intensity color scheme highlights reflectivity
m4.add_lidar_layer(
    source="https://s3.amazonaws.com/hobu-lidar/autzen-classified.copc.laz",
    name="autzen-intensity",
    color_scheme="intensity",
    point_size=2,
    pickable=True,
)
m4